# Analisando os dados da COVID-19
> Uma análise exploratória de séries temporais

- toc: true 
- badges: true
- hide_binder_badge: true
- comments: true
- categories: [colab]
- image: images/chart-preview.png

Este post reúne análises das ocorrências de COVID-19 em diferentes países. Especificamente, os gráficos desta análise isolam países da Europa Ocidental, EUA, Japão e Irã, além do Brasil. Todos os países apresentados nesta análise tinham mais casos confirmados do que o Brasil no dia 18/03/2020. Os dados utilizados para análise são fornecidos pelo [Universidade John Hopkins](https://github.com/CSSEGISandData/COVID-19).

A análise se divide em função de fatores relacionados ao tempo e à população. Comparando diferentes tipos de gráfico, é possível entender o impacto destes fatores. Em cada gráfico, é possível visualizar informações sobre cada curva (e pontos específicos nesta curva) passando o mouse por cima do ponto desejado. Também é possível clicar nos nomes apresentados na legenda ao lado para esconder ou apresentar a curva de um determinado país. Várias outras opções estão disponíveis, podendo ser exploradas pelo menu que aparece no canto superior direito de cada gráfico.

> Sempre que alternar entre tipos de condição (confirmado, morte ou recuperado), clique no botão que reseta os eixos (ícone de uma casa no menu superior direito de cada gráfico).

In [0]:
#hide 
import datetime

import pandas as pd
import plotly.express as px

# base variables
initial_date = datetime.date(2020,1,22)
days_elapsed = (datetime.date.today() - initial_date).days
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"

# dicts for data collection
dates = (f"{(initial_date + datetime.timedelta(days=days)).strftime('%m-%d-%Y')}" for days in range(0, days_elapsed))
csvs = {f"{day}": f"{base_url}/{day}.csv" for day in dates}

# collecting data
keep_columns = "Country/Region,Confirmed,Deaths,Recovered".split(",")
dfs = pd.concat(pd.read_csv(url)[keep_columns].assign(date=day) for day, url in csvs.items() if day < "03-23-2020")
dfs.columns = ["country", "confirmed", "deaths", "recovered", "date"]

keep_columns2 = "Country_Region,Confirmed,Deaths,Recovered".split(",")
dfs2 = pd.concat(pd.read_csv(url)[keep_columns2].assign(date=day) for day, url in csvs.items() if day >= "03-23-2020")
dfs2.columns = ["country", "confirmed", "deaths", "recovered", "date"]

dfs = pd.concat([dfs, dfs2])
dfs.index = range(0, len(dfs))

# fixing inconsistencies
dfs.loc[dfs["country"] == 'Iran (Islamic Republic of)', "country"] = "Iran"
dfs.loc[dfs["country"] == 'Republic of Korea', "country"] = "Korea, South"

# removing empty entries
all_na = dfs["confirmed"].isna() & dfs["deaths"].isna() & dfs["recovered"].isna()
df_imputed = dfs[~all_na]

## Usando contagem de ocorrências em valores absolutos

### Casos confirmados por dia

In [2]:
#hide 
# aggregating occurrences
df_aggregated = df_imputed.pivot_table(index=["date", "country"], 
                                       values=["confirmed", "deaths", "recovered"], 
                                       aggfunc='sum').reset_index()

# filtering countries
countries = ['Italy', 'Iran', 'Spain', 'Germany', 'France', 'US', 
             'Switzerland', 'United Kingdom', 'Japan', 'Portugal', 'Brazil']
# df_countries = df_aggregated.query("date == '03-22-2020'").sort_values("confirmed", ascending=False).set_index("country")
# countries = list(df_countries.loc[:"Brazil"].index)
df_brazil = df_aggregated.query(f"country in {countries} and country != 'China'")

# resampling time series
df_brazil["date"] = pd.to_datetime(df_brazil["date"])
df_brazil = df_brazil.groupby("country").resample("D", on="date").sum().reset_index()
ts_brazil = pd.melt(df_brazil, ["country", "date"], var_name="status", value_name="count")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
#hide_input
po = px.line(ts_brazil, x="date", y="count", color="country", animation_frame="status", height=600, width=1000)
po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dia",
    yaxis_title="Condição",
    updatemenus=[{"visible": False}],
)

### Casos confirmados em dias desde o primeiro caso

In [4]:
#hide 

# computing day zero
start_date = df_imputed.groupby("country")["date"].min().reset_index()
start_date.columns = ["country", "start_date"]

# adding day zero to df
df_extended = pd.merge(start_date, df_imputed)
days_elapsed = pd.to_datetime(df_extended["date"]) - pd.to_datetime(df_extended["start_date"])
df_extended["days"] = days_elapsed.astype('timedelta64[D]')
df_extended[["confirmed","deaths","recovered"]] = df_extended[["confirmed","deaths","recovered"]].fillna(0)

# aggregating occurrences
df_aggregated_day0 = df_extended.pivot_table(index=["date","days","country"],
                                             values=["confirmed", "deaths", "recovered"], 
                                             aggfunc='sum').reset_index()

# filtering countries
df_brazil_day0 = df_aggregated_day0.query(f"country in {countries} and country != 'China'")

# resampling time series
df_brazil_day0["date"] = pd.to_datetime(df_brazil_day0["date"])
df_brazil_day0 = df_brazil_day0.groupby("country").resample("D", on="date").sum().reset_index()
ts_brazil_day0 = pd.melt(df_brazil_day0.drop("date", axis=1), ["country", "days"], var_name="status", value_name="count")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
#hide_input
po = px.line(ts_brazil_day0, x="days", y="count", color="country", animation_frame="status",  height=600, width=1000)
po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dias desde a confirmação do 1o caso",
    yaxis_title="Condição",
    updatemenus=[{"visible": False}],
)

### Casos confirmados desde o primeiro dia com 100 casos confirmados

In [0]:
#hide

# removing entries with less than 100 ocurrences
ts_100_confirmed = df_brazil_day0.query("confirmed >= 100")

# computing start day
start_day = ts_100_confirmed.groupby("country")["days"].min().reset_index()
start_day.columns = ["country", "start_day"]

# updating start day
df_100_confirmed = pd.merge(start_day, ts_100_confirmed)
df_100_confirmed["days100"] = df_100_confirmed["days"] - df_100_confirmed["start_day"]
df_100_confirmed = df_100_confirmed.drop(["start_day","days"], axis = 1)
ts_100_confirmed = pd.melt(df_100_confirmed.drop("date", axis=1), ["country", "days100"], var_name="status", value_name="count")

In [7]:
#hide_input
po = px.line(ts_100_confirmed, x="days100", y="count", color="country", animation_frame="status", height=600, width=1000)
po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dias desde a confirmação do 100o caso",
    yaxis_title="Condição",
    updatemenus=[{"visible": False}],
)

## Normalizando os dados em relação ao tamanho da população

In [0]:
#hide
pwt = pd.read_excel("https://www.rug.nl/ggdc/docs/pwt91.xlsx", sheet_name="Data")

In [0]:
#hide 
data_pop = pwt.loc[pwt["year"] == 2017, ["countrycode", "country", "pop"]]
data_pop["pop"] = data_pop["pop"] * 1000000
data_pop = data_pop.set_index("countrycode")

data_pop.loc["USA", "country"] = "US"
data_pop.loc["IRN", "country"] = "Iran"
data_pop.loc["KOR", "country"] = "Korea, South"

df_relative = pd.merge(df_brazil, data_pop)
df_relative["confirmed"] = (df_relative["confirmed"] / df_relative["pop"]) * 100000
df_relative["deaths"] = (df_relative["deaths"] / df_relative["pop"]) * 100000
df_relative["recovered"] = (df_relative["recovered"] / df_relative["pop"]) * 100000
ts_relative = pd.melt(df_relative.drop("pop", axis=1), 
                      ["country", "date"], var_name="status", value_name="count")

### Casos confirmados por 100 mil habitantes por dia

In [11]:
#hide_input
po = px.line(ts_relative, x="date", y="count", color="country", animation_frame="status", height=600, width=1000)

po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dia",
    yaxis_title="Condição (por 100 mil habitantes)",
    updatemenus=[{"visible": False}],
)

#### Casos confirmados  por 100 mil habitantes em dias desde o primeiro caso

In [0]:
#hide 
df_relative_day0 = pd.merge(df_brazil_day0, data_pop)
df_relative_day0["confirmed"] = (df_relative_day0["confirmed"] / df_relative_day0["pop"]) * 100000
df_relative_day0["deaths"] = (df_relative_day0["deaths"] / df_relative_day0["pop"]) * 100000
df_relative_day0["recovered"] = (df_relative_day0["recovered"] / df_relative_day0["pop"]) * 100000
ts_relative_day0 = pd.melt(df_relative_day0.drop(["date","pop"], axis=1), 
                           ["country", "days"], var_name="status", value_name="count")

In [14]:
#hide_input
po = px.line(ts_relative_day0, x="days", y="count", color="country", animation_frame="status", height=600, width=1000)

po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dias desde a confirmação do 1o caso",
    yaxis_title="Condição (por 100 mil habitantes)",
    updatemenus=[{"visible": False}],
)

#### Casos confirmados  por 100 mil habitantes desde o primeiro dia com 100 casos confirmados

In [0]:
#hide
df_relative_100_confirmed = pd.merge(df_100_confirmed, data_pop)
df_relative_100_confirmed["confirmed"] = (df_relative_100_confirmed["confirmed"] / df_relative_100_confirmed["pop"]) * 100000
df_relative_100_confirmed["deaths"] = (df_relative_100_confirmed["deaths"] / df_relative_100_confirmed["pop"]) * 100000
df_relative_100_confirmed["recovered"] = (df_relative_100_confirmed["recovered"] / df_relative_100_confirmed["pop"]) * 100000
ts_relative_100_confirmed = pd.melt(df_relative_100_confirmed.drop(["date", "pop"], axis=1), 
                                    ["country", "days100"], var_name="status", value_name="count")

In [16]:
#hide_input
po = px.line(ts_relative_100_confirmed, x="days100", y="count", color="country", animation_frame="status", height=600, width=1000)
po.update_layout(
    title='Evolução dos casos de COVID-19 nos países que atualmente tem mais casos que o Brasil',
    xaxis_title="Dias desde a confirmação do 100o caso",
    yaxis_title="Condição (por 100 mil habitantes)",
    updatemenus=[{"visible": False}],
)